# ELVIS Mini Example - Factory Methods

This notebook demonstrates how to use ELVIS factory methods for quick EV charging simulation setup.

## Setup

Import required modules and set up the environment:

In [ ]:
import os
import sys
import datetime

# Add project root to Python path
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import ELVIS components
from elvis.config import ScenarioConfig
from elvis.simulate import simulate

## Basic Factory Method Usage

Create pre-configured scenarios with a single line:

In [ ]:
# Create different scenario types
office_config = ScenarioConfig.office_scenario()
residential_config = ScenarioConfig.residential_scenario()
gasstation_config = ScenarioConfig.gasstation_speedway_scenario()

print("Factory method scenarios created:")
print(f"Office scenario: {office_config.num_charging_events} events")
print(f"Residential scenario: {residential_config.num_charging_events} events")
print(f"Gas station scenario: {gasstation_config.num_charging_events} events")

## Customizing Factory Methods

Easily modify scenarios by passing parameters:

In [ ]:
# Customize number of charging events
office_small = ScenarioConfig.office_scenario(num_charging_events=50)

# Change scheduling policy
office_fcfs = ScenarioConfig.office_scenario(scheduling_policy='FCFS')

# Multiple customizations
office_custom = ScenarioConfig.office_scenario(
    num_charging_events=25,
    scheduling_policy='DISCRIMINATION_FREE'
)

print("Customized scenarios:")
print(f"Small office: {office_small.num_charging_events} events")
print(f"FCFS policy: {office_fcfs.scheduling_policy}")
print(f"Custom: {office_custom.num_charging_events} events, {office_custom.scheduling_policy} policy")

## Running a Complete Simulation

From scenario creation to results analysis:

In [ ]:
# 1. Create scenario
config = ScenarioConfig.office_scenario(num_charging_events=10)

# 2. Define simulation period
start_date = "2025-01-01 08:00:00"
end_date = "2025-01-07 18:00:00"  # One week
resolution = "01:00:00"  # 1 hour resolution

# 3. Create realisation and simulate
realisation = config.create_realisation(start_date, end_date, resolution)
results = simulate(realisation, print_progress=False)
results.scenario = realisation

# 4. Calculate metrics
start_dt = datetime.datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
end_dt = datetime.datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S")
num_steps = int((end_dt - start_dt) / datetime.timedelta(hours=1)) + 1

results.aggregate_load_profile(num_steps)

print("Simulation Results:")
print(f"Total energy charged: {results.total_energy_charged(num_steps):.2f} kWh")
print(f"Maximum load: {results.max_load():.2f} kW")
print(f"Average charging time: {results.average_charging_time().total_seconds()/3600:.2f} hours")
print(f"Number of charging events: {len(realisation.charging_events)}")

## Comparing Scheduling Policies

Easily test different charging control strategies:

In [ ]:
policies = ['UNCONTROLLED', 'FCFS', 'DISCRIMINATION_FREE']
results_comparison = {}

# Simulation parameters
start_date = "2025-01-01 00:00:00"
end_date = "2025-01-03 23:00:00"  # Short 3-day test
resolution = "01:00:00"

for policy in policies:
    print(f"Running {policy} simulation...")
    
    # Create scenario with specific policy
    config = ScenarioConfig.office_scenario(
        num_charging_events=20,
        scheduling_policy=policy
    )
    
    # Simulate
    realisation = config.create_realisation(start_date, end_date, resolution)
    result = simulate(realisation, print_progress=False)
    result.scenario = realisation
    
    # Calculate metrics
    start_dt = datetime.datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
    end_dt = datetime.datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S")
    num_steps = int((end_dt - start_dt) / datetime.timedelta(hours=1)) + 1
    
    result.aggregate_load_profile(num_steps)
    
    results_comparison[policy] = {
        'total_energy': result.total_energy_charged(num_steps),
        'max_load': result.max_load(),
        'avg_charge_time': result.average_charging_time().total_seconds() / 3600
    }

# Display results
print("\nScheduling Policy Comparison:")
print("Policy".ljust(20) + "Energy (kWh)".ljust(15) + "Max Load (kW)".ljust(15) + "Avg Time (h)")
print("-" * 65)
for policy, metrics in results_comparison.items():
    print(f"{policy:<20}{metrics['total_energy']:<15.2f}{metrics['max_load']:<15.2f}{metrics['avg_charge_time']:<15.2f}")

## Available Factory Methods

ELVIS provides 12 pre-configured scenarios:

In [ ]:
factory_methods = [
    "office_scenario",
    "residential_scenario", 
    "wohnblock_scenario",
    "cabstand_scenario",
    "customer_parking_day_scenario",
    "customer_parking_night_scenario",
    "gasstation_speedway_scenario",
    "kundenparkplatz_scenario",
    "office_fleet_scenario",
    "pnr_scenario",
    "roadside_scenario",
    "tankstelle_city_scenario"
]

print("Available factory methods:")
for method in factory_methods:
    print(f"  - ScenarioConfig.{method}()")

print(f"\nTotal: {len(factory_methods)} predefined scenarios")

## Key Benefits

Factory methods provide:

1. **Quick Start**: Create scenarios with one line of code
2. **Validation**: Pre-validated, working configurations
3. **Customization**: Easy parameter modification
4. **Consistency**: Standardized scenarios for research
5. **Research Ready**: Focus on analysis, not configuration

For advanced customization beyond factory method parameters, you can still access all ScenarioConfig methods and properties after creation.